This program use SymPy to symbolically derive several quantities that are described
in the manuscript: the (1) relaxation and buoyancy transfer functions, (2) the velocity
solutions, and (3) a limiting value of one of the eigenvalues of the problem.

This file is best read/used in conjunction with the derivations in the manuscript.

## Elevation Solutions

In [1]:
import sympy as sp
import matplotlib.pyplot as plt

k = sp.Symbol('k')          # scaled wavenumber (H*wavenumber, where H=ice thickness)  
expk = sp.exp(k)
# use this matrix for floating ice:
M = sp.Matrix(( [expk, -1/expk, k*expk,-k/expk], [expk, 1/expk, expk*(k+1),(k-1)/expk], [1, 1, 1,-1],[1,-1,0,0] ))

b1 = sp.Symbol('b1')                # proportional to -h
b2 = sp.Symbol('b2')                # proportional to delta*s
e0 = sp.Symbol('e0')                # constant 
e1 = sp.Symbol('e1')                # constant 

Delta = sp.Symbol('Delta')


# solution vector
A,B,C,D = sp.symbols('A,B,C,D')

# rhs vector:
b = sp.Matrix(4,1,[b1,-e0*k*b1,e1*k*b2,b2])

sol, = sp.linsolve((M,b),[A,B,C,D])

# vertical velocity at upper surface of ice sheet (modulo 1/k factor)
w_h = expk*sol[0] + (1/expk)*sol[1] + k*expk*sol[2] + (k/expk)*sol[3]


Print relaxation and buoyancy transfer functions
(the extension-related terms have the e_0 and e_1 on them):

In [2]:
# # print the formulas (modulo a 1/k factor):

# print the coefficient on h
R = sp.collect(sp.collect(sp.simplify(w_h.subs(b1,1).subs(b2,0)),expk),e0)
print('R = ')
sp.pprint(R)

# print the coefficient on delta*s
B = sp.collect(sp.collect(sp.simplify(-w_h.subs(b1,0).subs(b2,1)),e1),expk)
print('\n \n B = ')
sp.pprint(B)

R = 
⎛        3      ⎞  2⋅k    4⋅k    
⎝- 4⋅e₀⋅k  - 4⋅k⎠⋅ℯ    - ℯ    + 1
─────────────────────────────────
    ⎛   2    ⎞  2⋅k    4⋅k       
    ⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1   

 
 B = 
  ⎛   ⎛   2  2⋅k    2⎞                 2⋅k    ⎞  k
2⋅⎝e₁⋅⎝- k ⋅ℯ    + k ⎠ - k + (-k - 1)⋅ℯ    + 1⎠⋅ℯ 
──────────────────────────────────────────────────
            ⎛   2    ⎞  2⋅k    4⋅k                
            ⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1            


In [3]:
# # print the formulas (modulo a 1/k factor):
w_b = sol[0]+sol[1]

# print the coefficient on h
B = sp.collect(sp.collect(sp.simplify(w_b.subs(b1,1).subs(b2,0)),e0),expk)
print('B = ')
sp.pprint(B)

# print the coefficient on delta*s
R = sp.collect(sp.collect(sp.simplify(-w_b.subs(b1,0).subs(b2,1)),e1),expk)
print('\n \n R = ')
sp.pprint(R)

B = 
  ⎛   ⎛   2  2⋅k    2⎞                 2⋅k    ⎞  k
2⋅⎝e₀⋅⎝- k ⋅ℯ    + k ⎠ - k + (-k - 1)⋅ℯ    + 1⎠⋅ℯ 
──────────────────────────────────────────────────
            ⎛   2    ⎞  2⋅k    4⋅k                
            ⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1            

 
 R = 
⎛        3      ⎞  2⋅k    4⋅k    
⎝- 4⋅e₁⋅k  - 4⋅k⎠⋅ℯ    - ℯ    + 1
─────────────────────────────────
    ⎛   2    ⎞  2⋅k    4⋅k       
    ⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1   


## Velocity Solutions

We also want the vertical velocity at an arbitrary depth z, which we can compute as follows:

In [4]:
#---------------------- 2. VELOCITY SOLUTIONS------------------------------------
z = sp.Symbol('z')          # depth

# w(z) modulo a H/k factor
w = sp.exp(k*z)*sol[0] + (1/sp.exp(k*z))*sol[1] + k*z*sp.exp(k*z)*sol[2] + k*z*sol[3]/sp.exp(k*z)

print('h component:')
sp.pprint(sp.simplify(w.subs(b2,0).subs(b1,1).subs(e1,0).subs(e0,0)))
print('\n s component:')
sp.pprint(sp.simplify(w.subs(b1,0).subs(b2,1).subs(e1,0).subs(e0,0)))

h component:
⎛      ⎛     2⋅k    2⋅k    ⎞      2⋅k       ⎛    ⎛       2⋅k    ⎞      2⋅k    
⎝- k⋅z⋅⎝2⋅k⋅ℯ    + ℯ    - 1⎠ - k⋅ℯ    - k + ⎝k⋅z⋅⎝2⋅k + ℯ    - 1⎠ - k⋅ℯ    - k
──────────────────────────────────────────────────────────────────────────────
                                                2  2⋅k    4⋅k      2⋅k        
                                             4⋅k ⋅ℯ    - ℯ    + 2⋅ℯ    - 1    

    2⋅k    ⎞  2⋅k⋅z    2⋅k    ⎞  -k⋅z + k
 - ℯ    + 1⎠⋅ℯ      - ℯ    + 1⎠⋅ℯ        
─────────────────────────────────────────
                                         
                                         

 s component:
⎛⎛     2       ⎛       2⋅k    ⎞          2⋅k    ⎞  2⋅k   ⎛   2  2⋅k       ⎛   
⎝⎝- 2⋅k  + k⋅z⋅⎝2⋅k + ℯ    - 1⎠ + 2⋅k + ℯ    - 1⎠⋅ℯ    + ⎝2⋅k ⋅ℯ    - k⋅z⋅⎝2⋅k
──────────────────────────────────────────────────────────────────────────────
                                                      2  2⋅k    4⋅k      2⋅k  
                                                  

Now we will compute the horizontal velocity expressions:

In [5]:
# coefficients from the elevation solution problem
sol = sol.subs(e1,0).subs(e0,0) # set extensional terms to zero
A,B,C,D = sol

alpha = sp.Symbol('alpha')          #ik_x

wb = (A+B)/k                                    # w at z=0
wh = (A*expk + B/expk + k*expk*C + k*D/expk)/k        # w at z=H

wz0 = A-B+C+D                                   # dw/dz at z=0

wzh = A*expk -B/expk +C*expk +C*k*expk -D*k/expk + D/expk # dw/dz at z=H

# second derivative at z=H
wzzh = A*k*expk + B*k/expk + 2*C*k*expk + C*k*k*expk + D*k*k/expk  -2*D*k/expk

# second derivative at z=0
wzz0 = k*(A+B+2*C-2*D)

Ph = wzh - wz0*(expk+1/expk)/2- wzz0*(1/k)*(expk-1/expk)/2   # P(H)

Pzh = wzzh - wz0*k*(expk-1/expk)/2 - wzz0*(expk+1/expk)/2    # P_z(H)

b3 = -(k*wh + Pzh/k)*(alpha/k**2)                                 # first rhs vector entry

b4 = -(k*wb)*(alpha/k**2)                                          # 2nd rhs vector entry

# Matrix for horizontal surface velocity solutions
M2 = sp.Matrix(( [expk, -1/expk],[1, -1]))

# solution vector
E,F = sp.symbols('E,F')

# RHS vector:
d = sp.Matrix(2,1,[b3,b4])

sol2, = sp.linsolve((M2,d),[E,F])



wz = A*sp.exp(k*z) - B*sp.exp(-k*z) + C*(1+k*z)*sp.exp(k*z)+ D*(1-k*z)*sp.exp(-k*z)

coshkz = (sp.exp(k*z) + sp.exp(-k*z))/2
sinhkz = (sp.exp(k*z) - sp.exp(-k*z))/2


P = wz -wz0*coshkz- wzz0*sinhkz/k

u = P*(alpha/k**2)  + sol2[0]*sp.exp(k*z) + sol2[1]*sp.exp(-k*z)

## print velocity response functions
print('h component:')
sp.pprint(sp.collect(sp.collect(sp.simplify(-u.subs(b2,0).subs(b1,1).subs(alpha,1)),expk),1-z))
print('\n s component:')
sp.pprint(sp.collect(sp.collect(sp.simplify(u.subs(b1,0).subs(b2,1).subs(alpha,1)),expk),1-z))

h component:
⎛         2⋅k⋅z      2⋅k⋅z      2⋅k⋅(z + 1)                         2⋅k    2⋅k
⎝- 2⋅k⋅z⋅ℯ      + z⋅ℯ      - z⋅ℯ            + z + (-2⋅k⋅z - z + 1)⋅ℯ    - ℯ   
──────────────────────────────────────────────────────────────────────────────
                                           ⎛⎛   2    ⎞  2⋅k    4⋅k    ⎞       
                                         k⋅⎝⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1⎠       

⋅z    2⋅k⋅(z + 1)    ⎞  k⋅(1 - z)
   + ℯ            - 1⎠⋅ℯ         
─────────────────────────────────
                                 
                                 

 s component:
⎛         2⋅k⋅(z + 1)        2⋅k⋅(z + 1)      4⋅k      2⋅k⋅z      2⋅k⋅(z + 1) 
⎝- 2⋅k⋅z⋅ℯ            + 2⋅k⋅ℯ            - z⋅ℯ    + z⋅ℯ      - z⋅ℯ            
──────────────────────────────────────────────────────────────────────────────
                                          ⎛⎛   2    ⎞  2⋅k    4⋅k    ⎞        
                                        k⋅⎝⎝4⋅k  + 2⎠⋅ℯ    - ℯ    - 1⎠        

          

## Eigenvalue Limit

Finally we compute the limit of the larger eigenvalue in the problem as $k\to 0$, see the next notebook or the manuscript for more context. 

In [6]:
d = sp.Symbol('delta',positive=True)
R0 = (expk**4)+4*k*(expk**2)-1
D = k*(expk**4-2*(1+2*k**2)*(expk**2)+1)
B0 = 2*(k+1)*(expk**3) + 2*(k-1)*expk
R = R0/D
B = B0/D

# symbolic expression for the larger eigenvalue in the problem:
lamda = -(d+1)*R*(1- sp.sqrt((4*d/(d+1)**2)*((B0/R0)**2  -1) + 1))/2

# take the limit as k --> 0:
L = sp.limit(lamda,k,0)

# # print the limit:
print('limit of lambda_+ as k goes to zero:')
sp.pprint(sp.factor(sp.simplify(L)))

limit of lambda_+ as k goes to zero:
   -δ    
─────────
2⋅(δ + 1)
